# Eat Safe, Love

## Notebook Set Up

In [104]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [105]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [106]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [107]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [108]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [109]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusinessName': 1, 'scores.Hygiene': 1}

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print(f"Total documents: {document_count}")

# Display the first document in the results using pprint
results = establishments.find(query, fields)
pprint(results[0])

Total documents: 41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('676db06dd198cfc08d961e77'),
 'scores': {'Hygiene': 20}}


In [110]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df)}")

# Apply a lambda function to extract the BusinessName and Hygiene score
df["HygieneScore"] = df["scores"].apply(lambda x: x["Hygiene"])

# Drop the scores column from the DataFrame
df.drop(columns=["scores"], inplace=True)

# Display the first 10 rows of the DataFrame
df.head(10)

Number of rows: 41


,_id,BusinessName,HygieneScore
0,676db06dd198cfc08d961e77,The Chase Rest Home,20
1,676db06dd198cfc08d9621f8,Brenalwood,20
2,676db06dd198cfc08d962502,Melrose Hotel,20
3,676db06dd198cfc08d9626f3,Seaford Pizza,20
4,676db06dd198cfc08d962700,Golden Palace,20
5,676db06dd198cfc08d9630a2,Ashby's Butchers,20
6,676db06dd198cfc08d9632bf,South Sea Express Cuisine,20
7,676db06dd198cfc08d9647ef,Golden Palace,20
8,676db06dd198cfc08d964c30,The Tulip Tree,20
9,676db06dd198cfc08d965440,F & S,20


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [111]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': {'$regex':'London','$options':'i'},
         'RatingValue': {'$gte': 4}}
fields = {'BusinessName': 1, 'RatingValue': 1}

# Use count_documents to display the number of documents in the result
documents = establishments.count_documents(query)
print(f"Total documents: {documents}")

# Display the first document in the results using pprint
results = establishments.find(query, fields)
pprint(results[0])

Total documents: 33
{'BusinessName': "Charlie's",
 'RatingValue': 4,
 '_id': ObjectId('676db06dd198cfc08d963891')}


In [112]:
# Convert the result to a Pandas DataFrame
df_rating = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df_rating)}")

# Display the first 10 rows of the DataFrame
df_rating.head(10)

Number of rows: 33


,_id,BusinessName,RatingValue
0,676db06dd198cfc08d963891,Charlie's,4
1,676db06dd198cfc08d963bb7,Mv City Cruises Erasmus,5
2,676db06dd198cfc08d964705,Benfleet Motor Yacht Club,4
3,676db06dd198cfc08d965505,Coombs Catering t/a The Lock and Key,5
4,676db06dd198cfc08d965506,Tilbury Seafarers Centre,5
5,676db06dd198cfc08d965c0d,Mv Valulla,5
6,676db06dd198cfc08d967d1c,Tereza Joanne,5
7,676db06dd198cfc08d9680e3,Brick Lane Brews,4
8,676db06dd198cfc08d9680fc,The Nuance Group (UK) Limited,5
9,676db06dd198cfc08d9680fe,WH Smith,5


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [113]:
# Geolocation of Penang Flavours from the previous document
# 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'}

# Search within 0.01 degree on either side of the latitude and longitude.
degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    'geocode.latitude': {
        '$gte': float(latitude) - degree_search,
        '$lte': float(latitude) + degree_search
    },
    'geocode.longitude': {
        '$gte': float(longitude) - degree_search,
        '$lte': float(longitude) + degree_search
    },
    # Rating value must equal 5
    'RatingValue': 5
}

# Sort by hygiene score
sort = [('scores.Hygiene', 1)]
limit = 5

# Print the results
hygiene_results = establishments.find(query, { "BusinessName":1, "RatingValue":1, "scores.Hygiene":1}).sort(sort).limit(limit)
for result in hygiene_results:
    pprint(result)

{'BusinessName': 'Volunteer',
 'RatingValue': 5,
 '_id': ObjectId('676db06dd198cfc08d9676d6'),
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Plumstead Manor Nursery',
 'RatingValue': 5,
 '_id': ObjectId('676db06dd198cfc08d9676ee'),
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Atlantic Fish Bar',
 'RatingValue': 5,
 '_id': ObjectId('676db06dd198cfc08d9676f0'),
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Iceland',
 'RatingValue': 5,
 '_id': ObjectId('676db06dd198cfc08d9676aa'),
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Howe and Co Fish and Chips - Van 17',
 'RatingValue': 5,
 '_id': ObjectId('676db06dd198cfc08d9676b9'),
 'scores': {'Hygiene': 0}}


In [124]:
# Convert result to Pandas DataFrame
hygiene_results = establishments.find(query, {"BusinessName": 1, "RatingValue": 1, "scores.Hygiene": 1}).sort(sort).limit(limit)
df_hygiene = pd.DataFrame(hygiene_results)

# Extract the Hygiene score from the scores field and drop the scores field
df_hygiene["HygieneScore"] = df_hygiene["scores"].apply(lambda x: x["Hygiene"])
df_hygiene.drop(columns=["scores"], inplace=True)

# Display the first 5 rows of the DataFrame
df_hygiene.head(5)

,_id,BusinessName,RatingValue,HygieneScore
0,676db06dd198cfc08d9676d6,Volunteer,5,0
1,676db06dd198cfc08d9676ee,Plumstead Manor Nursery,5,0
2,676db06dd198cfc08d9676f0,Atlantic Fish Bar,5,0
3,676db06dd198cfc08d9676aa,Iceland,5,0
4,676db06dd198cfc08d9676b9,Howe and Co Fish and Chips - Van 17,5,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [126]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
match_query = {"$match":{"scores.Hygiene": 0}}

# 2. Groups the matches by Local Authority
group_query = {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}}

# 3. Sorts the matches from highest to lowest
sort_query = {"$sort": {"count": -1}}

# Print the number of documents in the result
pipeline = [match_query, group_query, sort_query]
result = list(establishments.aggregate(pipeline))
print(f"Total documents: {len(result)}")

# Print the first 10 results
pprint(result[:10])


Total documents: 55
[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [127]:
# Convert the result to a Pandas DataFrame
df_local_authority = pd.DataFrame(result)

# Display the number of rows in the DataFrame
print(f"Number of rows: {len(df_local_authority)}")

# Display the first 10 rows of the DataFrame
df_local_authority.head(10)

Number of rows: 55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
